In [5]:
import torch
import torch.nn as nn

class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        # Input gate layers
        self.Wii = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.Whi = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bi = nn.Parameter(torch.Tensor(hidden_size))

        # Forget gate layers
        self.Wif = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.Whf = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bf = nn.Parameter(torch.Tensor(hidden_size))

        # Output gate layers
        self.Wio = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.Who = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bo = nn.Parameter(torch.Tensor(hidden_size))

        # Cell state layers
        self.Wig = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.Whg = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bg = nn.Parameter(torch.Tensor(hidden_size))

        self.init_weights()

    def init_weights(self):
        for p in self.parameters():
            if p.data.ndimension() >= 2:
                nn.init.xavier_uniform_(p.data)
            else:
                nn.init.zeros_(p.data)

    def forward(self, x, init_states=None):
        """Assumes x is of shape (batch, input_size)"""
        batch_size, _ = x.size()

        hidden_seq = []

        if init_states is None:
            h_t, c_t = (torch.zeros(batch_size, self.hidden_size).to(x.device),
                        torch.zeros(batch_size, self.hidden_size).to(x.device))
        else:
            h_t, c_t = init_states

        # LSTM Cell calculations
        i_t = torch.sigmoid(x @ self.Wii + h_t @ self.Whi + self.bi)
        f_t = torch.sigmoid(x @ self.Wif + h_t @ self.Whf + self.bf)
        g_t = torch.tanh(x @ self.Wig + h_t @ self.Whg + self.bg)
        o_t = torch.sigmoid(x @ self.Wio + h_t @ self.Who + self.bo)

        c_t = f_t * c_t + i_t * g_t
        h_t = o_t * torch.tanh(c_t)

        # h_t is also the output
        return (h_t, c_t)

# Example usage
input_size = 10
hidden_size = 20
seq_len = 5
batch_size = 3

model = LSTMCell(input_size, hidden_size)

# Creating a dummy input sequence of shape (seq_len, batch, input_size)
x = torch.randn(seq_len, batch_size, input_size)
init_states = (torch.zeros(batch_size, hidden_size),
               torch.zeros(batch_size, hidden_size))

# Processing the input sequence
outputs = []
for i in range(seq_len):
    init_states = model(x[i], init_states)
    outputs.append(init_states[0])

# Final output for the sequence
final_outputs = torch.stack(outputs)

print(final_outputs.shape)


torch.Size([5, 3, 20])
